# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 192 (delta 12), reused 14 (delta 6), pack-reused 172 (from 1)
Receiving objects: 100% (192/192), 49.88 MiB | 25.53 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_L24O_cv
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

2025-04-29 21:06:07.215652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745960767.460539      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745960767.527129      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg + el preprocesamiento del artículo)

In [3]:
import requests
import pickle
import io
from sklearn.preprocessing import OneHotEncoder


url = 'https://raw.githubusercontent.com/dannasalazar11/Msc_thesis/main/gmrrnet_adhd/L24SO/results/X_preprocessed.pkl'
response = requests.get(url)
X = pickle.load(io.BytesIO(response.content))

url = 'https://raw.githubusercontent.com/dannasalazar11/Msc_thesis/main/gmrrnet_adhd/L24SO/results/y.pkl'
response = requests.get(url)
y = pickle.load(io.BytesIO(response.content))
# y = y.reshape(-1, 1)  # sklearn espera 2D
# encoder = OneHotEncoder(sparse_output=False)
# y = encoder.fit_transform(y)

_, _, sbjs = get_segmented_data()

X.shape, y.shape, len(sbjs)

((8213, 836), (8213,), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC


def automatic_identification(X,y):
    # Run ANOVA
    selector = SelectKBest(score_func=f_classif, k='all')
    selector.fit(X, y)
    
    # Get p-values and scores
    p_values = selector.pvalues_
    scores = selector.scores_
    
    # Create mask for p-values <= 0.5
    mask = p_values <= 0.5
    
    # Apply the mask to X
    X_selected = X[:, mask]
    
    print("Selected features:", np.sum(mask))
    print("X shape after ANOVA filter:", X_selected.shape)

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)
    
    # Apply PCA
    pca = PCA(n_components=0.90)  # Retain 90% of the variance
    X_pca = pca.fit_transform(X_scaled)
    
    # Print results
    print("Original ANOVA-selected shape:", X_selected.shape)
    print("Reduced shape after PCA:", X_pca.shape)
    print("Number of components selected:", X_pca.shape[1])

    return X_pca, SVC(kernel='rbf', gamma='scale', C=1.0)    



In [5]:
model_name = 'Automatic_Identification'

X_pca, model = automatic_identification(X,y)

Selected features: 756
X shape after ANOVA filter: (8213, 756)
Original ANOVA-selected shape: (8213, 756)
Reduced shape after PCA: (8213, 177)
Number of components selected: 177


# Resultados - Leave 24 Subjects Out

In [6]:
import os

import pickle

with open("/kaggle/input/ieee-tdah-control-database/folds.pkl", "rb") as f:
    folds = pickle.load(f)

In [7]:
results = train_L24O_cv(model, X_pca, y, sbjs, folds=folds, model_name=model_name)

Fold 1/5. Test subjects: ['v28p', 'v274', 'v1p', 'v231', 'v22p', 'v29p', 'v206', 'v238', 'v31p', 'v35p', 'v177', 'v200', 'v112', 'v113', 'v48p', 'v140', 'v131', 'v125', 'v55p', 'v143', 'v43p', 'v305', 'v134', 'v114']
Fold metrics: {'accuracy': 0.6691832532601235, 'recall': 0.6694439409730089, 'precision': 0.6691086967172031, 'kappa': 0.3382651974724431, 'auc': 0.6694439409730089}
Fold 2/5. Test subjects: ['v18p', 'v39p', 'v234', 'v32p', 'v190', 'v6p', 'v254', 'v204', 'v24p', 'v183', 'v246', 'v219', 'v298', 'v41p', 'v47p', 'v308', 'v52p', 'v300', 'v59p', 'v299', 'v302', 'v51p', 'v109', 'v127']
Fold metrics: {'accuracy': 0.6642685851318945, 'recall': 0.6378939206904644, 'precision': 0.678005610209, 'kappa': 0.2883427730312629, 'auc': 0.6378939206904644}
Fold 3/5. Test subjects: ['v215', 'v3p', 'v209', 'v37p', 'v213', 'v15p', 'v284', 'v181', 'v19p', 'v34p', 'v263', 'v244', 'v138', 'v121', 'v46p', 'v54p', 'v120', 'v310', 'v147', 'v50p', 'v56p', 'v107', 'v297', 'v108']
Fold metrics: {'accur

In [8]:
import numpy as np

accs = []
for result in results:
    accs.append(result['accuracy'])

np.mean(accs)

0.6646673541545063

In [9]:
import pickle

with open(f'results_L24SO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)